In [61]:
import torch
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn.functional as F

import torch.nn as nn
import matplotlib.pyplot as plt
import wandb
from tqdm.notebook import tqdm


In [55]:
root_train = '/dtu/datasets1/02514/hotdog_nothotdog/train/'
root_test = '/dtu/datasets1/02514/hotdog_nothotdog/test/'
batch_size = 32

transform = transforms.Compose([
    transforms.Resize((250,250)),
    transforms.ToTensor(),
])

train_set = datasets.ImageFolder(root=root_train, transform=transform)
test_set = datasets.ImageFolder(root=root_test, transform=transform)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [56]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1), # 
            nn.ReLU(),
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

In [57]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Network()
model.to(device)
#Initialize the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

#Get the first minibatch
data = next(iter(train_loader))[0].cuda()

#Try running the model on a minibatch
model(data); #if this runs the model dimensions fit

In [62]:
num_epochs = 5

for epoch in tqdm(range(num_epochs), unit='epoch'):
    #For each epoch
    train_correct = 0
    model.train()
    for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
        data, target = data.to(device), target.to(device)
        
        #Zero the gradients computed for each weight
        optimizer.zero_grad()

        #Forward pass your image through the network
        output = model(data)

        #Compute the loss
        loss = F.nll_loss(torch.log(output), target)

        #Backward pass through the network
        loss.backward()

        #Update the weights
        optimizer.step()
        
        #Compute how many were correctly classified
        predicted = output.argmax(1)
        train_correct += (target==predicted).sum().cpu().item()
    #Comput the test accuracy
    test_correct = 0
    model.eval()
    for data, target in test_loader:
        data = data.to(device)
        with torch.no_grad():
            output = model(data)
        predicted = output.argmax(1).cpu()
        test_correct += (target==predicted).sum().item()
    train_acc = train_correct/len(train_set)
    test_acc = test_correct/len(test_set)
    print("Accuracy train: {train:.1f}%\t test: {test:.1f}%".format(test=100*test_acc, train=100*train_acc))

  0%|          | 0/5 [00:00<?, ?epoch/s]

  0%|          | 0/64 [00:00<?, ?it/s]

RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of size: : [32]